In [1]:
import os
import cv2
import numpy as np
from google.colab import drive
import albumentations as A

# Mount Google Drive
drive.mount('/content/drive')

# Path dataset
path = "/content/drive/MyDrive/pmk_dataset/yolo/lidah/train"

Mounted at /content/drive


In [2]:
# Target jumlah foto per kelas
TARGET_PHOTOS = 30

# Definisi augmentasi (ringan untuk data medis)
def get_augmentation():
    return A.Compose([
        A.OneOf([
            A.Rotate(limit=15, p=0.8),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.8),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.GaussianBlur(blur_limit=3, p=0.3),
        ], p=1.0)
    ])

In [3]:
# Fungsi untuk augmentasi per kelas
def augment_class(class_folder_path, class_name, target_count):
    print(f"\n=== Memproses Kelas {class_name} ===")

    # Dapatkan semua file gambar
    image_files = [f for f in os.listdir(class_folder_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    original_count = len(image_files)
    needed_count = target_count - original_count

    print(f"Foto asli: {original_count}")
    print(f"Foto dibutuhkan: {needed_count}")

    if needed_count <= 0:
        print("Sudah cukup, tidak perlu augmentasi!")
        return

    # Siapkan augmentasi
    transform = get_augmentation()

    # Hitung berapa kali harus mengulang
    augmentations_per_image = needed_count // original_count
    extra_augmentations = needed_count % original_count

    print(f"Augmentasi per foto: {augmentations_per_image}")
    print(f"Augmentasi tambahan: {extra_augmentations}")

    counter = 1

    # Lakukan augmentasi
    for i, image_file in enumerate(image_files):
        # Baca gambar
        image_path = os.path.join(class_folder_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Nama dasar untuk file augmented
        base_name = os.path.splitext(image_file)[0]

        # Augmentasi reguler
        for j in range(augmentations_per_image):
            augmented = transform(image=image)['image']
            augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

            # Simpan dengan nama yang dinomori
            new_name = f"{base_name}_{counter:02d}.jpg"
            save_path = os.path.join(class_folder_path, new_name)
            cv2.imwrite(save_path, augmented_bgr)

            counter += 1
            print(f"  Dibuat: {new_name}")

        # Augmentasi tambahan (jika diperlukan)
        if i < extra_augmentations:
            augmented = transform(image=image)['image']
            augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

            new_name = f"{base_name}_{counter:02d}.jpg"
            save_path = os.path.join(class_folder_path, new_name)
            cv2.imwrite(save_path, augmented_bgr)

            counter += 1
            print(f"  Dibuat: {new_name}")

    print(f"Selesai! Total foto sekarang: {len(os.listdir(class_folder_path))}")

In [4]:
# Jalankan augmentasi untuk semua kelas
def main():
    # Install albumentations jika belum ada
    try:
        import albumentations
    except ImportError:
        print("Installing albumentations...")
        os.system("pip install albumentations")
        import albumentations as A

    print("=== MULAI AUGMENTASI DATA PMK ===")
    print(f"Target: {TARGET_PHOTOS} foto per kelas")

    # Loop untuk setiap kelas (0, 1, 2, 3)
    classes = ['0', '1', '2', '3']

    for class_name in classes:
        class_path = os.path.join(path, class_name)

        if os.path.exists(class_path):
            augment_class(class_path, class_name, TARGET_PHOTOS)
        else:
            print(f"Folder kelas {class_name} tidak ditemukan!")

    print("\n=== AUGMENTASI SELESAI ===")

    # Tampilkan summary
    print("\n=== SUMMARY ===")
    for class_name in classes:
        class_path = os.path.join(path, class_name)
        if os.path.exists(class_path):
            total_files = len([f for f in os.listdir(class_path)
                             if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            print(f"Kelas {class_name}: {total_files} foto")

# Jalankan program
if __name__ == "__main__":
    main()

=== MULAI AUGMENTASI DATA PMK ===
Target: 30 foto per kelas

=== Memproses Kelas 0 ===
Foto asli: 12
Foto dibutuhkan: 18
Augmentasi per foto: 1
Augmentasi tambahan: 6


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: main-qimg-5905a43ae85f80dfe71fe289124490d0-lq_01.jpg
  Dibuat: main-qimg-5905a43ae85f80dfe71fe289124490d0-lq_02.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.32 PM_03.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.32 PM_04.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.30 PM (1)_05.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.30 PM (1)_06.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.32 PM (2)_07.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.32 PM (2)_08.jpg
  Dibuat: IMG_20221105_104630_09.jpg
  Dibuat: IMG_20221105_104630_10.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.30 PM_11.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.30 PM_12.jpg
  Dibuat: istockphoto-826256132-612x612_13.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.31 PM_14.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.33 PM (1)_15.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.29 PM (1)_16.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.53.31 PM (1)_17.jpg
  Dibuat: WhatsApp Image 2022-11-10 at 7.

/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: lidah_5_01.jpg
  Dibuat: lidah_5_02.jpg
  Dibuat: lidah_5_03.jpg
  Dibuat: lidah_5_04.jpg
  Dibuat: lidah_5_05.jpg
  Dibuat: lidah_5_06.jpg
  Dibuat: lidah_5_07.jpg
  Dibuat: lidah_6_08.jpg
  Dibuat: lidah_6_09.jpg
  Dibuat: lidah_6_10.jpg
  Dibuat: lidah_6_11.jpg
  Dibuat: lidah_6_12.jpg
  Dibuat: lidah_6_13.jpg
  Dibuat: lidah_6_14.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_15.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_16.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_17.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_18.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_19.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_20.jpg
  Dibuat: lidah_1_21.jpg
  Dibuat: lidah_1_22.jpg
  Dibuat: lidah_1_23.jpg
  Dibuat: lidah_1_24.jpg
  Dibuat: lidah_1_25.jpg
  Dibuat: lidah_1_26.jpg
Selesai! Total foto sekarang: 30

=== Memproses Kelas 2 ===
Foto asli: 3
Foto dibutuhkan: 27
Augmentasi per foto: 9
Augmentasi tambahan: 0


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: lidah_4_01.jpg
  Dibuat: lidah_4_02.jpg
  Dibuat: lidah_4_03.jpg
  Dibuat: lidah_4_04.jpg
  Dibuat: lidah_4_05.jpg
  Dibuat: lidah_4_06.jpg
  Dibuat: lidah_4_07.jpg
  Dibuat: lidah_4_08.jpg
  Dibuat: lidah_4_09.jpg
  Dibuat: image-14_10.jpg
  Dibuat: image-14_11.jpg
  Dibuat: image-14_12.jpg
  Dibuat: image-14_13.jpg
  Dibuat: image-14_14.jpg
  Dibuat: image-14_15.jpg
  Dibuat: image-14_16.jpg
  Dibuat: image-14_17.jpg
  Dibuat: image-14_18.jpg
  Dibuat: lidah_3_19.jpg
  Dibuat: lidah_3_20.jpg
  Dibuat: lidah_3_21.jpg
  Dibuat: lidah_3_22.jpg
  Dibuat: lidah_3_23.jpg
  Dibuat: lidah_3_24.jpg
  Dibuat: lidah_3_25.jpg
  Dibuat: lidah_3_26.jpg
  Dibuat: lidah_3_27.jpg
Selesai! Total foto sekarang: 30

=== Memproses Kelas 3 ===
Foto asli: 5
Foto dibutuhkan: 25
Augmentasi per foto: 5
Augmentasi tambahan: 0


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: lidah_2p_01.jpg
  Dibuat: lidah_2p_02.jpg
  Dibuat: lidah_2p_03.jpg
  Dibuat: lidah_2p_04.jpg
  Dibuat: lidah_2p_05.jpg
  Dibuat: lidah_2_06.jpg
  Dibuat: lidah_2_07.jpg
  Dibuat: lidah_2_08.jpg
  Dibuat: lidah_2_09.jpg
  Dibuat: lidah_2_10.jpg
  Dibuat: images_11.jpg
  Dibuat: images_12.jpg
  Dibuat: images_13.jpg
  Dibuat: images_14.jpg
  Dibuat: images_15.jpg
  Dibuat: 5c5df14c69535_16.jpg
  Dibuat: 5c5df14c69535_17.jpg
  Dibuat: 5c5df14c69535_18.jpg
  Dibuat: 5c5df14c69535_19.jpg
  Dibuat: 5c5df14c69535_20.jpg
  Dibuat: FMD_010_21.jpg
  Dibuat: FMD_010_22.jpg
  Dibuat: FMD_010_23.jpg
  Dibuat: FMD_010_24.jpg
  Dibuat: FMD_010_25.jpg
Selesai! Total foto sekarang: 30

=== AUGMENTASI SELESAI ===

=== SUMMARY ===
Kelas 0: 30 foto
Kelas 1: 30 foto
Kelas 2: 30 foto
Kelas 3: 30 foto
